In [0]:
import collections
import sys
sys.path.append('./')
import pandas as pd
import numpy as np
import glob
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re
from config import mapping
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import utils
from sklearn.metrics import accuracy_score
import numpy as np

In [0]:
process = False
dir = '../data/Basketball/Team/gamelog/'
odds_data_path = '../data/scraped_odds_data.csv'

teams = ['ATL', 'BOS', 'BRK', 'CHI', 'CHO', 'CLE', 'DAL', 'DEN', 'DET',
         'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN',
         'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS',
         'TOR', 'UTA', 'WAS']

In [0]:
mapping = collections.defaultdict(int)

In [0]:
mapping = {
    'Philadelphia 76ers' : 'PHI',
    'Denver Nuggets': 'DEN',
    'Golden State Warriors' : 'GSW',
    'Milwaukee Bucks' : 'MIL',
    'Toronto Raptors' : 'TOR',
    'Los Angeles Clippers' : 'LAC',
    'San Antonio Spurs' : 'SAS',
    'Houston Rockets' : 'HOU',
    'Portland Trail Blazers' : 'POR',
    'Utah Jazz' : 'UTA',
    'Detroit Pistons' : 'DET',
    'Oklahoma City Thunder' : 'OKC',
    'Orlando Magic' : 'ORL',
    'Indiana Pacers' : 'IND',
    'Brooklyn Nets' : 'BRK',
    'Boston Celtics' : 'BOS',
    'Charlotte Hornets' : 'CHO',
    'Los Angeles Lakers' : 'LAL',
    'Sacramento Kings' : 'SAC',
    'Phoenix Suns' : 'PHO',
    'Dallas Mavericks' : 'DAL',
    'New Orleans Pelicans' : 'NOP',
    'Atlanta Hawks' : 'ATL',
    'Miami Heat' : 'MIA',
    'Washington Wizards' : 'WAS',
    'Minnesota Timberwolves' : 'MIN',
    'New York Knicks' : 'NYK',
    'Chicago Bulls' : 'CHI',
    'Memphis Grizzlies' : 'MEM',
    'Cleveland Cavaliers' : 'CLE',
}

In [0]:
import feather

In [0]:
X_test = feather.read_dataframe('../data/X_test_df.feather')
df = X_test

In [0]:
y_pred = feather.read_dataframe('../data/nn_test_predictions.feather')
y_test = feather.read_dataframe('../data/y_test_df.feather')

In [0]:
df.shape

(11931, 311)

In [0]:
dodd = pd.read_csv('../data/scraped_odds_data.csv', header = 0)


In [0]:
dodd = pd.read_csv('../data/scraped_odds_data.csv', header = 0)

In [0]:
monthmap = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}

In [0]:
def changetime(x):
    y = x.split(' ')
    z = y[2] + '-' + str(monthmap[y[1]]) + '-' + y[0]
    return z

In [0]:
dict1 = dict()

In [0]:
for i in range(2,14602):
    if (dodd.loc[i][2] in mapping) and (dodd.loc[i][3] in mapping):
        key = (changetime(dodd.loc[i][0]), mapping[dodd.loc[i][2]], mapping[dodd.loc[i][3]] )
        dict1[key] = i

In [0]:
df.shape

(11931, 1)

In [0]:
z = np.zeros((11931, 2))

In [0]:
for i in range(0,11931):
    key1 = (df.loc[i]['Date'], df.loc[i]['Home'], df.loc[i]['Opp'])
    key2 = (df.loc[i]['Date'], df.loc[i]['Opp'], df.loc[i]['Home'])
    if key1 in dict1:
        #print(i, key1)
        if dodd.loc[dict1[key1]][5] != '-':
            z[i][0] = dodd.loc[dict1[key1]][5]
            z[i][1] = dodd.loc[dict1[key1]][6]
    if key2 in dict1:
        #print(i, key2)
        if dodd.loc[dict1[key2]][5] != '-':
            z[i][0] = dodd.loc[dict1[key2]][5]
            z[i][1] = dodd.loc[dict1[key2]][6]

array([-168822.,   84593.])

In [0]:
mark = 0
profit = 0
for i in range(0,11931):
    if z[i][0] != 0:
        mark += 100
        if y_pred.loc[i]['predictions'] == True:
            if y_test.loc[i]['target'] == True:
                profit += -100*100/z[i][0]
            if y_test.loc[i]['target'] == False:
                profit -= 100
        if y_pred.loc[i]['predictions'] == False:
            if y_test.loc[i]['target'] == True:
                profit -= 100
            if y_test.loc[i]['target'] == False:
                profit += z[i][1]
        

In [0]:
profit

-49717.11722686858

In [0]:
mark

88000